Given a vector $v\in \{-1,+1\}^n$, we are interested in the set of $k$ orthogonal vectors $r_1,\ldots, r_k \in \{-1,+1\}^n$, that solves
$$\max_{\{r_i\}_k}\min_{i\in [k]}v^Tr_i$$

From Tramer et al., 2018, it was shown that if for all $i$, $v^Tr_i \geq \alpha  n$ for $\alpha \in (0,1)$, then $\alpha \leq 1/\sqrt{k}$. In other words, $OPT=n/\sqrt{k}$. Let's denote by $R$ the matrix formed by stacking $r_i$ vertically.

In [133]:
from scipy.linalg import hadamard
import numpy as np
import itertools

In [134]:
# dim
n = 2682030 # imagenet dim
# adv cone size < n
k = 100
# target vector
v = np.sign(np.random.randn(n))
OPT= n / np.sqrt(k)

#### Naive Construction

The following naive method (Tramer et al, 2018) can be used to achieve ~$n/k$, a factor of $\sqrt{k}$ worse than OPT, assuming $k$ divides $n$.

In [135]:
def chain_lol(lol):
    return list(itertools.chain(*lol))

def construct_idx(chunk_size, k, n):
    """a method to get 1d idxs for the R matrix used by the naive construction method
    """
    return filter(
        lambda x: x < n*k,
        chain_lol(
        [range(i*(chunk_size)+j,i*(chunk_size ) + chunk_size+j) for i,j in enumerate(range(0,n*k,n))]))

def naive_R(n, k):
    chunk_size = (n + k - 1) // k
    R = np.zeros((k, n))
    R.ravel()[construct_idx(chunk_size, k, n)] = v
    return R

In [136]:
R = naive_R(n,k)

In [137]:
R.dot(v), OPT 

(array([26821., 26821., 26821., 26821., 26821., 26821., 26821., 26821.,
        26821., 26821., 26821., 26821., 26821., 26821., 26821., 26821.,
        26821., 26821., 26821., 26821., 26821., 26821., 26821., 26821.,
        26821., 26821., 26821., 26821., 26821., 26821., 26821., 26821.,
        26821., 26821., 26821., 26821., 26821., 26821., 26821., 26821.,
        26821., 26821., 26821., 26821., 26821., 26821., 26821., 26821.,
        26821., 26821., 26821., 26821., 26821., 26821., 26821., 26821.,
        26821., 26821., 26821., 26821., 26821., 26821., 26821., 26821.,
        26821., 26821., 26821., 26821., 26821., 26821., 26821., 26821.,
        26821., 26821., 26821., 26821., 26821., 26821., 26821., 26821.,
        26821., 26821., 26821., 26821., 26821., 26821., 26821., 26821.,
        26821., 26821., 26821., 26821., 26821., 26821., 26821., 26821.,
        26821., 26821., 26821., 26751.]), 268203.0)

A factor of 10 worse for Imagenet. Let's consider another construction from Tramer et al., 2018.

#### Tight Randomized Construction with Regular Hadamard matrix

In [149]:
R  = hadamard(128)

In [155]:
adjust = np.ones(128)
adjust[64:] = -1

In [156]:
R.dot(adjust)

array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 128.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.])